# 04. Window + Summary Memory (하이브리드)

이 노트북에서는 **Window + Summary Memory** 방식을 LangChain과 LangGraph로 구현합니다.

**Window + Summary Memory란?**
- Window: 최근 N개 대화는 그대로 유지
- Summary: 오래된 대화는 요약하여 저장
- 최근 맥락 + 전체 맥락 모두 활용

**학습 목표:**
- 두 방식의 장점 결합
- LangGraph 복합 상태 관리
- 실용적인 메모리 시스템 구현

---

## 1. 패키지 설치

In [ ]:
!pip install langchain langchain-anthropic langgraph langgraph-checkpoint-sqlite -q
print("설치 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 16.8 MB/s eta 0:00:00
설치 완료!


---

## 2. 모델 선택

Claude 4.5 모델 중 하나를 선택하세요.

In [ ]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
# MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-haiku-4-5-20251001


---

## 3. API 키 설정

In [ ]:
import os

# 방법 1: 직접 입력
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

# 방법 2: Colab Secrets
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    print("Colab Secrets에서 API 키를 불러왔습니다!")
except:
    print("직접 API 키를 입력하세요.")

Colab Secrets에서 API 키를 불러왔습니다!


---

## 4. LangChain으로 Window+Summary Memory 구현

### 4.1 기본 구성

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# LLM 초기화
llm = ChatAnthropic(
    model=MODEL,
    max_tokens=500
)

# 요약용 LLM
summarizer_llm = ChatAnthropic(
    model=MODEL,
    max_tokens=300
)

# 하이브리드 메모리 설정
WINDOW_SIZE = 6          # 최근 6개 메시지 유지 (LLM에 전달)
SUMMARIZE_THRESHOLD = 4  # 윈도우 밖 메시지가 4개 이상이면 요약

# 메모리 상태
all_messages = []          # 전체 메시지 (요약 전까지 보관)
conversation_summary = ""  # 누적 요약

print(f"Window+Summary Memory 설정 완료!")
print(f"  윈도우 크기: {WINDOW_SIZE}개")
print(f"  요약 임계값: 윈도우 밖 {SUMMARIZE_THRESHOLD}개 이상 시 요약")

Window+Summary Memory 설정 완료!
  윈도우 크기: 6개
  요약 임계값: 윈도우 밖 4개 이상 시 요약


### 4.2 핵심 함수 정의

In [ ]:
def summarize_messages(messages: list, existing_summary: str) -> str:
    """
    메시지를 요약에 통합합니다.
    """
    if not messages:
        return existing_summary

    # 메시지 텍스트 변환
    conversation_text = ""
    for msg in messages:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        conversation_text += f"{role}: {msg.content}\n"

    if existing_summary:
        prompt = f"""기존 대화 요약:
{existing_summary}

추가된 대화:
{conversation_text}

위 내용을 통합하여 간결하게 요약해주세요.
중요한 정보(이름, 선호도, 사실 등)를 반드시 포함하세요.
3-5문장으로 요약하세요."""
    else:
        prompt = f"""다음 대화를 간결하게 요약해주세요.
중요한 정보를 포함하세요.
3-5문장으로 요약하세요.

대화:
{conversation_text}"""

    response = summarizer_llm.invoke([HumanMessage(content=prompt)])
    return response.content


def manage_memory(new_message: str, is_user: bool = True):
    """
    Window+Summary 메모리 관리.

    로직:
    1. 새 메시지를 all_messages에 추가
    2. 윈도우 밖 메시지가 SUMMARIZE_THRESHOLD 이상이면:
       - 윈도우 밖 메시지를 요약에 통합
       - 윈도우 내 메시지만 유지
    """
    global all_messages, conversation_summary

    # 메시지 추가
    if is_user:
        all_messages.append(HumanMessage(content=new_message))
    else:
        all_messages.append(AIMessage(content=new_message))

    # 윈도우 밖 메시지 수 계산
    overflow_count = len(all_messages) - WINDOW_SIZE

    # 윈도우 밖 메시지가 임계값 이상이면 요약
    if overflow_count >= SUMMARIZE_THRESHOLD:
        # 윈도우 밖 메시지 (요약 대상)
        messages_to_summarize = all_messages[:overflow_count]

        print(f"\n    [요약 트리거: 윈도우 밖 {overflow_count}개 메시지 → 요약 중...]")
        conversation_summary = summarize_messages(messages_to_summarize, conversation_summary)
        print(f"    [요약 완료]")

        # 윈도우 내 메시지만 유지
        all_messages = all_messages[-WINDOW_SIZE:]


def get_recent_window() -> list:
    """현재 윈도우 내 메시지 반환"""
    return all_messages[-WINDOW_SIZE:] if len(all_messages) > WINDOW_SIZE else all_messages


print("메모리 관리 함수 정의 완료!")

메모리 관리 함수 정의 완료!


### 4.3 Window+Summary 챗봇

In [ ]:
def chat_with_hybrid_memory(user_input: str) -> str:
    """
    Window+Summary 하이브리드 메모리 챗봇.

    컨텍스트 구성:
    - 시스템 프롬프트 + 대화 요약 (있으면)
    - 최근 윈도우 메시지
    """
    global conversation_summary

    # 사용자 메시지 추가 (메모리 관리)
    manage_memory(user_input, is_user=True)

    # 시스템 프롬프트 구성
    system_content = "당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."

    if conversation_summary:
        system_content += f"\n\n【이전 대화 요약】\n{conversation_summary}"

    # 메시지 구성 (시스템 + 윈도우)
    recent_window = get_recent_window()
    messages = [
        SystemMessage(content=system_content),
        *recent_window
    ]

    # LLM 호출
    response = llm.invoke(messages)

    # AI 응답 추가 (메모리 관리)
    manage_memory(response.content, is_user=False)

    return response.content


# 초기화
all_messages = []
conversation_summary = ""

print("="*50)
print("Window+Summary Memory 테스트")
print("="*50)

Window+Summary Memory 테스트


### 4.4 테스트: 장기 대화에서 정보 유지

In [ ]:
# 대화 테스트
conversations = [
    "안녕! 내 이름은 준혁이야.",
    "나는 개발자야.",
    "좋아하는 프로그래밍 언어는 파이썬이야.",
    "취미는 독서야.",
    "최근에 읽은 책은 '클린 코드'야.",
    "좋아하는 음식은 삼겹살이야.",
    "서울에 살고 있어.",
    "오늘 코딩 공부 뭐 할까?",
]

for i, msg in enumerate(conversations, 1):
    response = chat_with_hybrid_memory(msg)
    print(f"\n[{i}] 사용자: {msg}")
    print(f"    AI: {response[:80]}..." if len(response) > 80 else f"    AI: {response}")

print("\n" + "="*50)
print("요약된 정보 확인 (초반 정보)")
print("="*50)

response = chat_with_hybrid_memory("내 이름, 직업, 좋아하는 언어가 뭐라고 했지?")
print(f"\n사용자: 내 이름, 직업, 좋아하는 언어가 뭐라고 했지?")
print(f"AI: {response}")


[1] 사용자: 안녕! 내 이름은 준혁이야.
    AI: 안녕하세요, 준혁님! 만나서 반갑습니다. 😊

저는 당신을 도와드릴 준비가 되어 있는 AI 어시스턴트입니다. 무엇을 도와드릴까요?

[2] 사용자: 나는 개발자야.
    AI: 안녕하세요, 준혁님! 개발자이시군요. 멋있습니다! 👨‍💻

저도 개발 관련된 주제로 도움을 드릴 수 있습니다. 예를 들어:

- **프로그래밍 ...

[3] 사용자: 좋아하는 프로그래밍 언어는 파이썬이야.
    AI: 오, 파이썬이군요! 좋은 선택입니다. 👍

파이썬은 정말 많은 장점이 있습니다:

- **문법이 간결하고 읽기 쉬움** - 개발 속도가 빠름
-...

[4] 사용자: 취미는 독서야.
    AI: 아, 독서를 좋아하시는군요! 멋진 취미입니다. 📚

개발자이면서 독서를 하시다니 정말 좋습니다. 독서는 개발자에게도 많은 도움이 되거든요:

-...

    [요약 트리거: 윈도우 밖 4개 메시지 → 요약 중...]
    [요약 완료]

[5] 사용자: 최근에 읽은 책은 '클린 코드'야.
    AI: 오, '클린 코드'! 정말 훌륭한 책이네요! 📖

로버트 C. 마틴(Uncle Bob)의 명저인데, 개발자라면 꼭 읽어야 할 필독서입니다. 어떤...

[6] 사용자: 좋아하는 음식은 삼겹살이야.
    AI: 아, 삼겹살이군요! 👍 정말 맛있는 선택입니다! 🍖

개발자들은 밤샘 코딩할 때 삼겹살 같은 푸짐한 음식이 정말 필요하죠. 😄

준혁님의 관심사...

    [요약 트리거: 윈도우 밖 4개 메시지 → 요약 중...]
    [요약 완료]

[7] 사용자: 서울에 살고 있어.
    AI: 아, 서울에 사시는군요! 🏙️

준혁님의 프로필이 점점 그려지네요:
- 💻 **직업**: 개발자 (파이썬 사용)
- 📚 **취미**: 독서 ('...

[8] 사용자: 오늘 코딩 공부 뭐 할까?
    AI: 좋은 질문이네요! 🎯 오늘 파이썬 공부를 뭘 할지 결정하는 건 중요하니까요.

준혁님의 **현재 수준과 관심사

### 4.5 현재 메모리 상태 확인

In [ ]:
print("【현재 대화 요약】")
print("-" * 50)
print(conversation_summary if conversation_summary else "(없음)")

print(f"\n【전체 메시지 수】 {len(all_messages)}개")
print(f"【윈도우 내 메시지】")
print("-" * 50)
for i, msg in enumerate(get_recent_window(), 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"[{i}] {role}: {content}")

【현재 대화 요약】
--------------------------------------------------
# 준혁님 프로필 요약

**준혁**은 파이썬을 주로 사용하는 개발자이며, 최근 '클린 코드'를 읽고 코딩 원칙에 관심을 가지고 있다. 취미는 독서이고 좋아하는 음식은 삼겹살이다. 파이썬 관련 기술 상담, 코드 리뷰, 프로젝트 설계 등 다양한 개발 지원을 필요로 할 수 있다.

【전체 메시지 수】 6개
【윈도우 내 메시지】
--------------------------------------------------
[1] 사용자: 서울에 살고 있어.
[2] AI: 아, 서울에 사시는군요! 🏙️

준혁님의 프로필이 점점 그려지네요:
- 💻 **직업**: ...
[3] 사용자: 오늘 코딩 공부 뭐 할까?
[4] AI: 좋은 질문이네요! 🎯 오늘 파이썬 공부를 뭘 할지 결정하는 건 중요하니까요.

준혁님의 *...
[5] 사용자: 내 이름, 직업, 좋아하는 언어가 뭐라고 했지?
[6] AI: 좋은 질문입니다! 🤔

이전 대화 요약에 따르면:

- 📝 **이름**: 준혁
- 💼 **...


---

## 5. LangGraph로 Window+Summary Memory 구현

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# 설정
WINDOW_SIZE_LG = 6
SUMMARIZE_THRESHOLD_LG = 4


# 커스텀 Reducer: 메시지 교체 (병합 아님)
def replace_messages(existing: list, new: list) -> list:
    """새 메시지 리스트로 교체 (add_messages와 다름)"""
    # new가 단일 메시지 추가인 경우
    if len(new) == 1 and len(existing) > 0:
        return existing + new
    # new가 전체 교체인 경우 (요약 후)
    return new


class HybridState(TypedDict):
    messages: Annotated[list, replace_messages]  # 커스텀 reducer
    summary: str
    pending_summarize: bool  # 요약 대기 플래그


def should_summarize(state: HybridState) -> str:
    """요약 필요 여부 판단"""
    overflow = len(state["messages"]) - WINDOW_SIZE_LG
    if overflow >= SUMMARIZE_THRESHOLD_LG:
        return "summarize"
    return "chatbot"


def summarize_node(state: HybridState) -> dict:
    """윈도우 밖 메시지를 요약에 통합"""
    messages = state["messages"]
    existing_summary = state.get("summary", "")

    overflow_count = len(messages) - WINDOW_SIZE_LG
    messages_to_summarize = messages[:overflow_count]
    window_messages = messages[-WINDOW_SIZE_LG:]

    # 텍스트 변환
    text = "\n".join([
        f"{'사용자' if isinstance(m, HumanMessage) else 'AI'}: {m.content}"
        for m in messages_to_summarize
    ])

    if existing_summary:
        prompt = f"기존 요약: {existing_summary}\n\n추가 대화:\n{text}\n\n통합 요약 (3-5문장, 중요 정보 포함):"
    else:
        prompt = f"대화 요약 (3-5문장, 중요 정보 포함):\n{text}"

    response = summarizer_llm.invoke([HumanMessage(content=prompt)])

    return {
        "summary": response.content,
        "messages": window_messages  # 윈도우만 유지 (replace_messages로 교체됨)
    }


def chatbot_node(state: HybridState) -> dict:
    """LLM 응답 생성"""
    system_content = "당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."

    summary = state.get("summary", "")
    if summary:
        system_content += f"\n\n【이전 대화 요약】\n{summary}"

    # 윈도우 내 메시지만 사용
    window = state["messages"][-WINDOW_SIZE_LG:]

    messages = [
        SystemMessage(content=system_content),
        *window
    ]

    response = llm.invoke(messages)

    # 새 메시지 추가 (기존 + 응답)
    return {"messages": state["messages"] + [response]}


# 그래프 구성
graph_builder = StateGraph(HybridState)
graph_builder.add_node("summarize", summarize_node)
graph_builder.add_node("chatbot", chatbot_node)

# 조건부 라우팅
graph_builder.add_conditional_edges(
    START,
    should_summarize,
    {"summarize": "summarize", "chatbot": "chatbot"}
)
graph_builder.add_edge("summarize", "chatbot")
graph_builder.add_edge("chatbot", END)

# 컴파일
memory_saver = MemorySaver()
graph = graph_builder.compile(checkpointer=memory_saver)

print("LangGraph Window+Summary Memory 설정 완료!")
print(f"  윈도우 크기: {WINDOW_SIZE_LG}개")
print(f"  요약 임계값: 윈도우 밖 {SUMMARIZE_THRESHOLD_LG}개 이상")

LangGraph Window+Summary Memory 설정 완료!
  윈도우 크기: 6개
  요약 임계값: 윈도우 밖 4개 이상


### 5.1 LangGraph 테스트

In [ ]:
config = {"configurable": {"thread_id": "hybrid-session-1"}}

def chat_with_langgraph_hybrid(user_input: str) -> str:
    """LangGraph 기반 Window+Summary 챗봇"""
    result = graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    return result["messages"][-1].content


print("="*50)
print("LangGraph Window+Summary Memory 테스트")
print("="*50)

# 여러 대화 진행
conversations = [
    "안녕! 내 이름은 준혁이야.",
    "나는 개발자야.",
    "좋아하는 프로그래밍 언어는 파이썬이야.",
    "취미는 독서야.",
    "최근에 읽은 책은 '클린 코드'야.",
    "오늘 코딩 공부 뭐 할까?",
]

for i, msg in enumerate(conversations, 1):
    response = chat_with_langgraph_hybrid(msg)
    print(f"\n[{i}] 사용자: {msg}")
    print(f"    AI: {response[:80]}..." if len(response) > 80 else f"    AI: {response}")

print("\n" + "="*50)
print("종합 정보 확인")
print("="*50)

response = chat_with_langgraph_hybrid("내 이름, 직업, 좋아하는 언어가 뭐라고 했지?")
print(f"\n[7] 사용자: 내 이름, 직업, 좋아하는 언어가 뭐라고 했지?")
print(f"    AI: {response}")

LangGraph Window+Summary Memory 테스트

[1] 사용자: 안녕! 내 이름은 준혁이야.
    AI: 안녕하세요, 준혁님! 👋 

만나뵙게 되어 반갑습니다. 저는 Claude라고 합니다. 무엇을 도와드릴까요?

[2] 사용자: 나는 개발자야.
    AI: 안녕하세요, 준혁님! 개발자시군요. 👨‍💻

저도 개발자분들과 함께 일하는 것을 즐깁니다. 어떤 분야의 개발을 주로 하시나요? 예를 들어:

-...

[3] 사용자: 좋아하는 프로그래밍 언어는 파이썬이야.
    AI: 좋은 선택입니다! 파이썬은 정말 훌륭한 언어네요. 🐍

파이썬이 좋은 이유:
- **배우기 쉽고** 직관적인 문법
- **다양한 분야**에서 활...

[4] 사용자: 취미는 독서야.
    AI: 오, 독서를 즐기시는군요! 📚 개발자면서 독서도 하시니 정말 좋은 조합입니다.

독서와 개발이 잘 어울리는 이유:
- **논리적 사고**를 키울...

[5] 사용자: 최근에 읽은 책은 '클린 코드'야.
    AI: 오, 《클린 코드》! 정말 명작이네요! 👍

Robert C. Martin의 《Clean Code》는 개발자라면 꼭 읽어야 할 필독서죠. 특히:...

[6] 사용자: 오늘 코딩 공부 뭐 할까?
    AI: 오늘의 코딩 공부 주제를 정하기 위해 몇 가지 여쭤보고 싶습니다! 🤔

**준혁님의 현재 상황:**
1. **현재 수준**은 어느 정도인가요?
...

종합 정보 확인

[7] 사용자: 내 이름, 직업, 좋아하는 언어가 뭐라고 했지?
    AI: 네, 맞습니다! 😊

**준혁님의 정보:**
- **이름**: 준혁
- **직업**: 개발자
- **좋아하는 언어**: 파이썬 🐍

이전 대화 요약에도 있었고, 방금 전 대화에서도 확인했네요!

혹시 제가 놓친 부분이 있나요? 아니면 추가로 알려주고 싶으신 정보가 있으신가요? 👀


### 5.2 저장된 상태 확인

In [ ]:
state = graph.get_state(config)

print("【대화 요약】")
print("-" * 50)
print(state.values.get("summary", "(없음)"))

print(f"\n【현재 윈도우】 ({len(state.values.get('window', []))}개)")
print("-" * 50)
for i, msg in enumerate(state.values.get('window', []), 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"[{i}] {role}: {content}")

print(f"\n【전체 메시지 수】 {len(state.values['messages'])}개")

【대화 요약】
--------------------------------------------------
## 대화 요약

준혁님은 개발자이며, 주로 파이썬을 사용하는 프로그래밍 언어입니다. Claude는 준혁님이 웹 개발, 모바일 앱, 게임 개발, 데이터 분석, DevOps, AI/머신러닝 등 다양한 분야에서 도움을 받을 수 있도록 제안했습니다. 준혁님은 코드 리뷰, 버그 해결, 알고리즘 설명, 학습 조언 등의 지원을 받을 수 있습니다.

【현재 윈도우】 (0개)
--------------------------------------------------

【전체 메시지 수】 9개


---

## 6. SQLite로 영구 저장

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

# SQLite 연결
conn = sqlite3.connect("hybrid_memory.db", check_same_thread=False)
sqlite_saver = SqliteSaver(conn)

print("SQLite 영구 저장 설정 완료!")

SQLite 영구 저장 설정 완료!


In [ ]:
# SQLite 테스트
config_sqlite = {"configurable": {"thread_id": "persistent-hybrid-1"}}

result = graph.invoke(
    {"messages": [HumanMessage(content="이 대화는 파일에 저장돼요!")]},
    config=config_sqlite
)

print("응답:", result["messages"][-1].content)
print("\nhybrid_memory.db에 저장됨!")

응답: 안녕하세요! 네, 저희 대화가 저장될 수 있다는 점을 알려주셔서 감사합니다. 

참고로 알려드리면:
- 저와의 대화는 **Claude의 개인정보 보호 정책**에 따라 처리됩니다
- 일반적으로 대화 내용이 저장될 수 있으며, 서비스 개선 등의 목적으로 사용될 수 있습니다
- 민감한 정보는 공유하지 않는 것이 좋습니다

혹시 특정한 질문이나 도움이 필요하신 것이 있으시면 편하게 말씀해 주세요! 😊

hybrid_memory.db에 저장됨!


---

## 7. 메모리 방식 비교 정리

| 방식 | 토큰 효율 | 맥락 유지 | 구현 복잡도 | 적합한 사용 사례 |
|------|----------|----------|------------|----------------|
| Buffer | 낮음 | 완벽 | 매우 쉬움 | 짧은 대화 |
| Window | 높음 | 최근만 | 쉬움 | 간단한 Q&A |
| Summary | 높음 | 좋음 | 중간 | 장기 대화 |
| Window+Summary | 높음 | 매우 좋음 | 높음 | 복잡한 작업 |

### Window+Summary 장점
- 최근 맥락: 윈도우에서 세부 사항 유지
- 장기 맥락: 요약에서 중요 정보 보존
- 토큰 효율: 일정 수준으로 유지

### 권장 설정
- 윈도우 크기: 4-6개 메시지 (2-3턴)
- 요약 임계값: 윈도우 크기의 1.5-2배

### Claude 4.5 모델 비교

| 모델 | ID | 특징 |
|------|-----|------|
| Haiku 4.5 | claude-haiku-4-5-20251001 | 빠름, 저렴 |
| Sonnet 4.5 | claude-sonnet-4-5-20250929 | 균형 |
| Opus 4.5 | claude-opus-4-5-20251101 | 최고 성능 |

---

## 학습 완료!

4가지 메모리 관리 방식을 모두 배웠습니다:
1. Buffer Memory - 전체 저장
2. Window Memory - 최근 N개
3. Summary Memory - 요약 저장
4. Window+Summary - 하이브리드

In [ ]:
# 정리
conn.close()
print("정리 완료!")

정리 완료!
